In [2]:
%pip install mistralai


   ---------------------------------------- 0/3 [invoke]
   -------------------------- ------------- 2/3 [mistralai]
   -------------------------- ------------- 2/3 [mistralai]
   -------------------------- ------------- 2/3 [mistralai]
   -------------------------- ------------- 2/3 [mistralai]
   -------------------------- ------------- 2/3 [mistralai]
   -------------------------- ------------- 2/3 [mistralai]
   -------------------------- ------------- 2/3 [mistralai]
   -------------------------- ------------- 2/3 [mistralai]
   -------------------------- ------------- 2/3 [mistralai]
   -------------------------- ------------- 2/3 [mistralai]
   ---------------------------------------- 3/3 [mistralai]

Note: you may need to restart the kernel to use updated packages.


In [6]:
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import dotenv

dotenv.load_dotenv(".env", override=True)

True

In [9]:
import os
from mistralai import Mistral

api_key = os.environ["MISTRAL_API_KEY"]

client = Mistral(api_key=api_key)

client.models.list().data

[BaseModelCard(id='mistral-medium-2505', capabilities=ModelCapabilities(completion_chat=True, completion_fim=False, function_calling=True, fine_tuning=True, vision=True, classification=False), object='model', created=1758794124, owned_by='mistralai', name='mistral-medium-2505', description='Our frontier-class multimodal model released May 2025.', max_context_length=131072, aliases=[], deprecation=None, deprecation_replacement_model=None, default_model_temperature=0.3, TYPE='base'),
 BaseModelCard(id='mistral-large-latest', capabilities=ModelCapabilities(completion_chat=True, completion_fim=False, function_calling=True, fine_tuning=True, vision=True, classification=False), object='model', created=1758794124, owned_by='mistralai', name='mistral-large-latest', description='Official mistral-large-latest Mistral AI model', max_context_length=131072, aliases=[], deprecation=None, deprecation_replacement_model=None, default_model_temperature=0.3, TYPE='base'),
 BaseModelCard(id='mistral-mediu

In [20]:
nikud_prompt = """
Add full Hebrew nikud (vowel marks) to the following text. 
Return only the hebrew text with nikud. Do not provide explanations or notes, or any other text other than the hebrew one. 
Text: """

In [62]:
# model = "codestral-2501"
model = "mistral-small-latest"
chat_response = client.chat.complete(
    model= model,
    n=5,
    temperature=0.6,
    messages = [
        {
            "role": "user",
            "content": nikud_prompt + "ממלכת רוהן הייתה ממלכה בדיונית שנוצרה על ידי ג'.ר.ר. טולקין בספרו \"שר הטבעות\".",
        },
    ]
)
print([choice.message.content for choice in chat_response.choices])

['מְמַלְכַּת רוֹהַן הָיְתָה מְמַלְכָּה בְּדִיּוֹנִית שֶׁנּוֹצְרָה עַל יְדֵי ג\'.ר.ר. טוֹלְקִין בְּסִפְרוֹ "שַׂר הַטַּבְעוֹת".', 'מְמַלְכַּת רוֹהָן הָיְתָה מְמַלְכָּה בְּדִיּוֹנִית שֶׁנִּצְרְתָה עַל יְדֵי ג\'.ר.ר. טוֹלְקִין בְּסִפְרוֹ "שַׂר הַטֶּבַע".', 'מְמַלְכַּת רוֹהָן הָיְתָה מְמַלְכָּה בְּדִיּוֹנִית שֶׁנִּצְרְעָה עַל יְדֵי ג\'.ר.ר. טוֹלְקִין בְּסִפְרוֹ "שַׂר הַטֶּבַע".', 'מְמַלְכַּת רוֹהָן הָיְתָה מְמַלְכָּה בְּדִיּוֹנִית שֶׁנִּצְרְעָה עַל יְדֵי ג\'.ר.ר. טוֹלְקִין בְּסִפְרוֹ "שַׂר הַטַּבְעָה".', 'מַמְלָכַת רוֹהַן הָיְתָה מַמְלָכָה בְּדִיּוֹנִית שֶׁנִּצְרְתָה עַל־יְדֵי ג\'.ר.ר. טוֹלְקִין בְּסִפְרוֹ "שַׂר הַטַּבְעוֹת".']


In [63]:
import pandas as pd

filtered_df = pd.read_csv('./datasets/hewiki/hebrew_nikud_dataset_filtered_word_mask.csv')
hewiki_sample = filtered_df.sample(1000)

In [64]:
import math
from collections import Counter

def compute_word_entropy_mask(original_sentence: str, outputs: list[str], threshold: float = 0.0):
    """
    Compute entropy-based ambiguity mask for words in a sentence.
    
    Args:
        original_sentence (str): input sentence without nikud (used for tokenization).
        outputs (list[str]): list of generated outputs with nikud (multiple generations).
        threshold (float): entropy threshold above which a word is marked ambiguous.
                           Default = 0.0 (any disagreement → ambiguous).
    
    Returns:
        mask (list[int]): 0/1 mask, length = number of words in original_sentence.
                          1 = ambiguous (uncertain), 0 = consistent (confident).
        entropies (list[float]): entropy per word (in bits).
    """
    # Split original text into words (no nikud, for alignment)
    words = original_sentence.split()
    n_words = len(words)

    # Collect nikud-versions per word across outputs
    word_variants = [ [] for _ in range(n_words) ]
    for out in outputs:
        out_words = out.split()
        if len(out_words) != n_words:
            # simple alignment fallback: skip misaligned outputs
            continue
        for i, w in enumerate(out_words):
            word_variants[i].append(w)

    entropies = []
    mask = []
    for variants in word_variants:
        if not variants:
            entropies.append(0.0)
            mask.append(0)
            continue
        counts = Counter(variants)
        total = sum(counts.values())
        probs = [c/total for c in counts.values()]
        entropy = -sum(p * math.log2(p) for p in probs)
        entropies.append(entropy)
        mask.append(1 if entropy > threshold else 0)

    return mask, entropies


In [65]:
import utils.hebrew_tokenizer as ht

def strip_nikud(text: str) -> str:
    """
    Remove Hebrew nikud (vowel marks) from the input text.
    
    Args:
        text (str): Input Hebrew text with nikud.
    """
    return ht.NIKUD_PATTERN.sub('', text)

In [ ]:
import os
import asyncio


async def generate_content_for_prompt(client: Mistral, prompt: str, n_outputs: int, model_name: str) -> list[str]:
    """
    An asynchronous helper function to call the API for a single prompt 
    and extract the generated text.
    """
    while True:
        try:
            # Use the asynchronous client module (client.aio)
            chat_response = await client.chat.complete_async(
                model=model_name,
                n=n_outputs,
                temperature=0.7,
                messages=[
                    {"role": "user", "content": prompt}
                ]
            )

            # print(f"Completed request")

            # Extract the text from each candidate
            results = [
                choice.message.content for choice in chat_response.choices]
            return results
        except Exception as e:
            if "429" in str(e):
                print(
                    f"Quota exceeded {e}. Waiting for 30 seconds before retrying...")
                await asyncio.sleep(30)  # Wait before retrying
                continue  # Retry the request
            # print(f"Request failed with error: {e}")
            # Return error message for a failed request
            raise RuntimeError(f"Request failed with error: {e}")


async def run_async_batch_prompts(prompts: list[str], n_outputs: int = 1, model_name: str = 'mistral-small-latest') -> list[list[str]]:
    """
    Sends a batch of text prompts to the Gemini API asynchronously and returns 
    the results as a list of lists.

    Args:
        prompts: A list of string prompts to send to the model.
        n_outputs: The number of distinct outputs (candidates) to generate for each prompt.
        model_name: The model to use for the batch job.

    Returns:
        A list of lists, where each inner list contains the n_outputs 
        generated texts for the corresponding input prompt.
    """
    if not os.getenv("MISTRAL_API_KEY"):
        raise RuntimeError("Error: MISTRAL_API_KEY environment variable is not set.")
    api_key = os.environ["MISTRAL_API_KEY"]

    client = Mistral(api_key=api_key)

    # print(
    #     f"Starting {len(prompts)} parallel requests, generating {n_outputs} candidates each...")

    # Create a list of all asynchronous tasks
    tasks = [
        generate_content_for_prompt(client, prompt, n_outputs, model_name)
        for prompt in prompts
    ]

    # Run all tasks concurrently and wait for them to complete
    results_list_of_lists = await asyncio.gather(*tasks)

    return results_list_of_lists

In [67]:
import time


async def process_batch_chunks(sample_df: pd.DataFrame, nikud_prompt: str, n_outputs: int = 5):
    """
    Asynchronous function to process prompts in chunks.

    Args:
        sample_df: The DataFrame containing the 'text' column to process.
        nikud_prompt: The prefix to add to each prompt.
        chunk_size: The number of prompts to send in one batch.
        n_outputs: The number of candidate responses to request per prompt.
    """
    # Prepare all prompts
    prompt_data = list(zip(sample_df['text'].index, [
                       nikud_prompt + strip_nikud(txt) for txt in sample_df['text'].tolist()]))
    sample_df['uncertainty_word_mask'] = pd.Series(
        [None] * len(sample_df), index=sample_df.index, dtype='object')

    for i in range(0, len(prompt_data)):
        start = time.time()
        index, prompt = prompt_data[i]

        res = await run_async_batch_prompts([prompt], n_outputs=n_outputs)

        uncertainty_word_mask = compute_word_entropy_mask(
            # Get the original text using the actual index
            sample_df.loc[index, 'text'],
            # Get the corresponding result from the batch response
            res[0],
            threshold=0.0
        )[0]

        # if 1 not in uncertainty_word_mask:
        #     print(
        #         f"Warning: No uncertain words found for index {index}. Mask: {uncertainty_word_mask}, Results: {res[0]}")
        sample_df.loc[index, 'uncertainty_word_mask'] = str(uncertainty_word_mask)
        
        elapsed = time.time() - start
        # If the request took less than 1s, wait the remainder
        if elapsed < 1:
            time.sleep(1 - elapsed)

    # return sample_df

In [ ]:
await process_batch_chunks(hewiki_sample, nikud_prompt)

Starting 1 parallel requests, generating 5 candidates each...
Starting 1 parallel requests, generating 5 candidates each...
Starting 1 parallel requests, generating 5 candidates each...
Starting 1 parallel requests, generating 5 candidates each...
Starting 1 parallel requests, generating 5 candidates each...
Starting 1 parallel requests, generating 5 candidates each...
Starting 1 parallel requests, generating 5 candidates each...
Starting 1 parallel requests, generating 5 candidates each...
Starting 1 parallel requests, generating 5 candidates each...
Starting 1 parallel requests, generating 5 candidates each...
Starting 1 parallel requests, generating 5 candidates each...
Starting 1 parallel requests, generating 5 candidates each...
Starting 1 parallel requests, generating 5 candidates each...
Starting 1 parallel requests, generating 5 candidates each...
